In [3]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [4]:
## Define train and test paths
train_path = './FRE2013/train'
test_path = './FRE2013/test'

In [5]:
from datetime import datetime
from packaging import version

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,MaxPool2D,Conv2D,Dropout,MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

In [20]:
%load_ext tensorboard
import tensorboard
tensorboard.__version__

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


'2.9.0'

In [ ]:
!rm -rf ./logs/

In [22]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [23]:
train_set=ImageDataGenerator().flow_from_directory(train_path,
                                             target_size=(48,48),
                                             batch_size=32,
                                             color_mode='grayscale',
                                             class_mode='categorical')
test_set=ImageDataGenerator().flow_from_directory(test_path,
                                             target_size=(48,48),
                                             batch_size=32,
                                             color_mode='grayscale',
                                             class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [39]:
## MODEL 1
model = Sequential()

model.add(BatchNormalization(input_shape=(48,48,1)))
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',input_shape=(48,48,1), padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Dropout(0.3))
model.add(Dense(1024))
model.add(Dense(7, activation='softmax'))

In [ ]:
# MODEL 2

model = Sequential()

model.add(BatchNormalization(input_shape=(48,48,1)))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(48,48,1),name="conv1"))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool1"))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation='relu',padding='same',name="conv2"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))         
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv4"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool4"))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024, activation='relu',name='fc1'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(7, activation='softmax',name='fcsoftmax'))

In [ ]:
# MODEL 3

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(48,48,1),name="conv1"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation='relu',padding='same',name="conv2"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))     
model.add(Dense(256, activation='relu'))    
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model.add(Dense(128, activation='relu'))  
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv4"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool4"))
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024, activation='relu',name='fc1'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(7, activation='softmax',name='fcsoftmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_39 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_24 (Dropout)        (None, 48, 48, 32)        0         
                                                                 
 conv2 (Conv2D)              (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_40 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 maxpool2 (MaxPooling2D)     (None, 24, 24, 32)      

In [ ]:
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [62]:
history=model.fit(train_set,epochs=300,validation_data=test_set,batch_size=16,callbacks=[tensorboard_callback])

Epoch 1/300
898/898 [==============================] - 38s 42ms/step - loss: 0.7889 - accuracy: 0.7074 - val_loss: 1.1067 - val_accuracy: 0.6144
Epoch 2/300
898/898 [==============================] - 38s 42ms/step - loss: 0.8359 - accuracy: 0.6873 - val_loss: 1.0929 - val_accuracy: 0.6053
Epoch 3/300
898/898 [==============================] - 38s 42ms/step - loss: 0.7703 - accuracy: 0.7137 - val_loss: 1.1229 - val_accuracy: 0.6094
Epoch 4/300
898/898 [==============================] - 38s 42ms/step - loss: 0.6909 - accuracy: 0.7446 - val_loss: 1.2178 - val_accuracy: 0.5954
Epoch 5/300
898/898 [==============================] - 38s 42ms/step - loss: 0.6882 - accuracy: 0.7419 - val_loss: 1.1604 - val_accuracy: 0.6137
Epoch 6/300
898/898 [==============================] - 38s 42ms/step - loss: 0.6195 - accuracy: 0.7689 - val_loss: 1.2361 - val_accuracy: 0.6081
Epoch 7/300
898/898 [==============================] - 38s 42ms/step - loss: 0.5628 - accuracy: 0.7934 - val_loss: 1.2103 - val_ac

KeyboardInterrupt: 

In [63]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets
